In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup as BS
import json
import seaborn as sns
import re
from IPython.core.display import HTML

# Rental Listings

In [2]:
gulfport = pd.read_csv('../data/gulfport_listings.csv')
dauphin = pd.read_csv('../data/dauphin_listings.csv')
miramar = pd.read_csv('../data/miramar_listings.csv')
orange = pd.read_csv('../data/orange_listings.csv')
palms = pd.read_csv('../data/palms_listings.csv')
panama = pd.read_csv('../data/panama_listings.csv')
tybee = pd.read_csv('../data/tybee_listings.csv')

In [3]:
gulfport = gulfport.drop(columns = 'Unnamed: 0')
dauphin = dauphin.drop(columns = 'Unnamed: 0')
miramar = miramar.drop(columns = 'Unnamed: 0')
orange = orange.drop(columns = 'Unnamed: 0')
palms = palms.drop(columns = 'Unnamed: 0')
panama = panama.drop(columns = 'Unnamed: 0')
tybee = tybee.drop(columns = 'Unnamed: 0')

In [4]:
miramar['location'] = 'Miramar Beach'
miramar = miramar.drop(columns = 'checkin_date')

In [5]:
tybee['location'] = 'Tybee Island'
tybee = tybee.drop(columns = 'checkin_date')

# Fixing Listings with 0 Rooms

In [6]:
for index, row in tybee.iterrows():
    if row.cost_night == 330:
        tybee.loc[index, 'bedrooms'] = 3
    elif row.cost_night == 1320:
        tybee.loc[index, 'bedrooms'] = 6
        
for index, row in orange.iterrows():
    if row.cost_night == 99:
        orange.loc[index, 'bedrooms'] = 2

for index, row in gulfport.iterrows():
    if row.cost_night == 140:
        gulfport.loc[index, 'bedrooms'] = 1

for index, row in dauphin.iterrows():
    if row.cost_night == 138:
        dauphin.loc[index, 'bedrooms'] = 1

for index, row in miramar.iterrows():
    if row.cost_night == 225:
        miramar.loc[index, 'bedrooms'] = 3
    elif row.cost_night == 190:
        miramar.loc[index, 'bedrooms'] = 1
    elif row.cost_night == 185:
        miramar.loc[index, 'bedrooms'] = 2
    elif row.cost_night == 204:
        miramar.loc[index, 'bedrooms'] = 2

for index, row in palms.iterrows():
    if row.cost_night == 375:
        palms.loc[index, 'bedrooms'] = 2
    elif row.cost_night == 512:
        miramar.loc[index, 'bedrooms'] = 2
    elif row.cost_night == 284:
        miramar.loc[index, 'bedrooms'] = 2
    elif row.cost_night == 517:
        miramar.loc[index, 'bedrooms'] = 2
    elif row.cost_night == 525:
        miramar.loc[index, 'bedrooms'] = 2

# Merging locations

In [7]:
all_locations = pd.merge(gulfport, dauphin,
                        on = ('bedrooms', 'type', 'sleeps', 'pets_allowed', 'rating', 'review_count',
       'cost_night', 'latitude', 'longitude', 'bathrooms', 'checkin_month',
       'checkin_day', 'location'), how='outer')
all_locations = pd.merge(all_locations, miramar,
                        on = ('bedrooms', 'type', 'sleeps', 'pets_allowed', 'rating', 'review_count',
       'cost_night', 'latitude', 'longitude', 'bathrooms', 'checkin_month',
       'checkin_day', 'location'), how='outer')
all_locations = pd.merge(all_locations, orange,
                        on = ('bedrooms', 'type', 'sleeps', 'pets_allowed', 'rating', 'review_count',
       'cost_night', 'latitude', 'longitude', 'bathrooms', 'checkin_month',
       'checkin_day', 'location'), how='outer')
all_locations = pd.merge(all_locations, palms,
                        on = ('bedrooms', 'type', 'sleeps', 'pets_allowed', 'rating', 'review_count',
       'cost_night', 'latitude', 'longitude', 'bathrooms', 'checkin_month',
       'checkin_day', 'location'), how='outer')
all_locations = pd.merge(all_locations, panama,
                        on = ('bedrooms', 'type', 'sleeps', 'pets_allowed', 'rating', 'review_count',
       'cost_night', 'latitude', 'longitude', 'bathrooms', 'checkin_month',
       'checkin_day', 'location'), how='outer')
all_locations = pd.merge(all_locations, tybee,
                        on = ('bedrooms', 'type', 'sleeps', 'pets_allowed', 'rating', 'review_count',
       'cost_night', 'latitude', 'longitude', 'bathrooms', 'checkin_month',
       'checkin_day', 'location'), how='outer')

# Rental Listing DF

In [8]:
all_locations = all_locations.drop(columns = 'checkin_day')
all_locations = all_locations.drop(columns = 'checkin_month')
all_locations

,bedrooms,type,sleeps,pets_allowed,rating,review_count,cost_night,latitude,longitude,bathrooms,location
0,3,House,9,True,4.888889,36,282,30.360629,-89.114292,2.0,Gulfport Beach
1,2,Condo,4,False,5.000000,3,175,30.369683,-89.085678,1.0,Gulfport Beach
2,3,House,6,False,4.777778,18,332,30.344905,-89.158187,2.5,Gulfport Beach
3,2,Condo,6,False,5.000000,7,185,30.390792,-89.003517,2.0,Gulfport Beach
4,3,House,10,True,4.864198,81,138,30.322899,-89.271076,2.0,Gulfport Beach
...,...,...,...,...,...,...,...,...,...,...,...
2216,2,Resort,8,False,0.000000,0,407,32.125060,-80.790669,2.0,Tybee Island
2217,2,Resort,6,False,0.000000,0,653,32.139866,-80.809869,2.0,Tybee Island
2218,2,Villa,8,False,5.000000,1,690,32.136659,-80.761516,2.0,Tybee Island
2219,6,House,13,False,0.000000,0,1225,32.110186,-80.815819,4.0,Tybee Island


# Pulling in weather

In [9]:
spring = pd.read_csv('../data/spring_weather_averages.csv')
summer = pd.read_csv('../data/summer_weather_avgs.csv')
fall = pd.read_csv('../data/fall_weather_totals.csv')

In [10]:
summer = summer.drop(columns = 'Unnamed: 0')
spring = spring.drop(columns = 'Unnamed: 0')
fall = fall.drop(columns = 'Unnamed: 0')

In [11]:
spring = spring[['location', 'month_num', 'day', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years']]
fall = fall[['location', 'month_num', 'day', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years']]
summer = summer[['location', 'month_num', 'day', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years']]

In [12]:
all_weather = pd.merge(spring, summer,
        on = ('location', 'month_num', 'day', 'temp', 'tempmax', 'tempmin',
       'days_with_rain_10_years'), how='outer')
all_weather = pd.merge(all_weather, fall,
        on = ('location', 'month_num', 'day', 'temp', 'tempmax', 'tempmin',
       'days_with_rain_10_years'), how='outer')

# Weather DF

In [13]:
all_weather

,location,month_num,day,temp,tempmax,tempmin,days_with_rain_10_years
0,Dauphin Island,3,11,64.28,71.55,57.59,70.0
1,Dauphin Island,3,12,58.88,67.56,51.99,40.0
2,Dauphin Island,3,13,57.71,66.63,49.02,30.0
3,Dauphin Island,3,14,59.06,68.54,50.06,30.0
4,Dauphin Island,3,15,60.34,69.55,51.57,40.0
...,...,...,...,...,...,...,...
968,Tybee Island,10,10,73.00,81.17,66.82,40.0
969,Tybee Island,10,11,73.03,81.42,66.14,30.0
970,Tybee Island,10,12,72.33,82.31,64.20,10.0
971,Tybee Island,10,13,73.04,82.87,65.22,20.0


# Grouping weather by check in dates

In [14]:
all_weather

,location,month_num,day,temp,tempmax,tempmin,days_with_rain_10_years
0,Dauphin Island,3,11,64.28,71.55,57.59,70.0
1,Dauphin Island,3,12,58.88,67.56,51.99,40.0
2,Dauphin Island,3,13,57.71,66.63,49.02,30.0
3,Dauphin Island,3,14,59.06,68.54,50.06,30.0
4,Dauphin Island,3,15,60.34,69.55,51.57,40.0
...,...,...,...,...,...,...,...
968,Tybee Island,10,10,73.00,81.17,66.82,40.0
969,Tybee Island,10,11,73.03,81.42,66.14,30.0
970,Tybee Island,10,12,72.33,82.31,64.20,10.0
971,Tybee Island,10,13,73.04,82.87,65.22,20.0


In [15]:
may_6 = all_weather.loc[(all_weather['month_num']==5)&(all_weather['day']>5)&(all_weather['day']<14)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
may_6['checkin_date'] = 'May 6'
may_6['checkout_date'] = 'May 13'
may_6['checkin_mon'] = 5
may_6['checkin_day'] = 6

may_13 = all_weather.loc[(all_weather['month_num']==5)&(all_weather['day']>12)&(all_weather['day']<21)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
may_13['checkin_date'] = 'May 13'
may_13['checkout_date'] = 'May 13'
may_13['checkin_mon'] = 5
may_13['checkin_day'] = 13

may_20 = all_weather.loc[(all_weather['month_num']==5)&(all_weather['day']>19)&(all_weather['day']<28)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
may_20['checkin_date'] = 'May 20'
may_20['checkout_date'] = 'May 20'
may_20['checkin_mon'] = 5
may_20['checkin_day'] = 20

june_3 = all_weather.loc[(all_weather['month_num']==6)&(all_weather['day']>2)&(all_weather['day']<11)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
june_3['checkin_date'] = 'June 3'
june_3['checkout_date'] = 'June 10'
june_3['checkin_mon'] = 6
june_3['checkin_day'] = 3

june_10 = all_weather.loc[(all_weather['month_num']==6)&(all_weather['day']>9)&(all_weather['day']<18)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
june_10['checkin_date'] = 'June 10'
june_10['checkout_date'] = 'June 17'
june_10['checkin_mon'] = 6
june_10['checkin_day'] = 10

june_17 = all_weather.loc[(all_weather['month_num']==6)&(all_weather['day']>16)&(all_weather['day']<25)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
june_17['checkin_date'] = 'June 17'
june_17['checkout_date'] = 'June 24'
june_17['checkin_mon'] = 6
june_17['checkin_day'] = 17

july_1 = all_weather.loc[(all_weather['month_num']==7)&(all_weather['day']>=1)&(all_weather['day']<9)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
july_1['checkin_date'] = 'July 1'
july_1['checkout_date'] = 'July 8'
july_1['checkin_mon'] = 7
july_1['checkin_day'] = 1

july_8 = all_weather.loc[(all_weather['month_num']==7)&(all_weather['day']>7)&(all_weather['day']<16)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
july_8['checkin_date'] = 'July 8'
july_8['checkout_date'] = 'July 15'
july_8['checkin_mon'] = 7
july_8['checkin_day'] = 8

july_15 = all_weather.loc[(all_weather['month_num']==7)&(all_weather['day']>14)&(all_weather['day']<23)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
july_15['checkin_date'] = 'July 15'
july_15['checkout_date'] = 'July 22'
july_15['checkin_mon'] = 7
july_15['checkin_day'] = 15

july_22 = all_weather.loc[(all_weather['month_num']==7)&(all_weather['day']>21)&(all_weather['day']<30)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
july_22['checkin_date'] = 'July 22'
july_22['checkout_date'] = 'July 29'
july_22['checkin_mon'] = 7
july_22['checkin_day'] = 22

august_5 = all_weather.loc[(all_weather['month_num']==8)&(all_weather['day']>4)&(all_weather['day']<13)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
august_5['checkin_date'] = 'August 5'
august_5['checkout_date'] = 'August 12'
august_5['checkin_mon'] = 8
august_5['checkin_day'] = 5

august_12 = all_weather.loc[(all_weather['month_num']==8)&(all_weather['day']>11)&(all_weather['day']<20)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
august_12['checkin_date'] = 'August 12'
august_12['checkout_date'] = 'August 19'
august_12['checkin_mon'] = 8
august_12['checkin_day'] = 12

august_19 = all_weather.loc[(all_weather['month_num']==8)&(all_weather['day']>18)&(all_weather['day']<27)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
august_19['checkin_date'] = 'August 19'
august_19['checkout_date'] = 'August 26'
august_19['checkin_mon'] = 8
august_19['checkin_day'] = 19

august_26 = all_weather.loc[(all_weather['month_num']==8)&(all_weather['day']>25)&(all_weather['day']<=31)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
august_26['checkin_date'] = 'August 26'
august_26['checkout_date'] = 'September 1'
august_26['checkin_mon'] = 8
august_26['checkin_day'] = 26

march_11 = all_weather.loc[(all_weather['month_num']==3)&(all_weather['day']>10)&(all_weather['day']<19)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
march_11['checkin_date'] = 'March 11'
march_11['checkout_date'] = 'March 18'
march_11['checkin_mon'] = 3
march_11['checkin_day'] = 11

october_7 = all_weather.loc[(all_weather['month_num']==10)&(all_weather['day']>=7)&(all_weather['day']<=14)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
october_7['checkin_date'] = 'October 7'
october_7['checkout_date'] = 'October 14'
october_7['checkin_mon'] = 10
october_7['checkin_day'] = 7

C:\Users\bswil\AppData\Local\Temp\ipykernel_25068\864220313.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  may_6 = all_weather.loc[(all_weather['month_num']==5)&(all_weather['day']>5)&(all_weather['day']<14)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
C:\Users\bswil\AppData\Local\Temp\ipykernel_25068\864220313.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  may_13 = all_weather.loc[(all_weather['month_num']==5)&(all_weather['day']>12)&(all_weather['day']<21)].groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
C:\Users\bswil\AppData\Local\Temp\ipykernel_25068\864220313.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  may_20 = all_w

In [16]:
may_27_1 = all_weather.loc[(all_weather['month_num']==5)&(all_weather['day']>26)]
may_27_2 = all_weather.loc[(all_weather['month_num']==6)&(all_weather['day']<=3)]
may_27 = pd.merge(may_27_1, may_27_2,
                 on = ('location', 'month_num', 'day', 'temp', 'tempmax', 'tempmin',
       'days_with_rain_10_years'), how = 'outer')
may_27 = may_27.groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
may_27['checkin_date'] = 'May 27'
may_27['checkout_date'] = 'June 3'
may_27['checkin_mon'] = 5
may_27['checkin_day'] = 27

june_24_1 = all_weather.loc[(all_weather['month_num']==6)&(all_weather['day']>=24)]
june_24_2 = all_weather.loc[(all_weather['month_num']==7)&(all_weather['day']<=1)]
june_24 = pd.merge(june_24_1, june_24_2,
                 on = ('location', 'month_num', 'day', 'temp', 'tempmax', 'tempmin',
       'days_with_rain_10_years'), how = 'outer')
june_24 = june_24.groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
june_24['checkin_date'] = 'June 24'
june_24['checkout_date'] = 'July 1'
june_24['checkin_mon'] = 6
june_24['checkin_day'] = 24

july_29_1 = all_weather.loc[(all_weather['month_num']==7)&(all_weather['day']>=29)]
july_29_2 = all_weather.loc[(all_weather['month_num']==8)&(all_weather['day']<=5)]
july_29 = pd.merge(july_29_1, july_29_2,
                 on = ('location', 'month_num', 'day', 'temp', 'tempmax', 'tempmin',
       'days_with_rain_10_years'), how = 'outer')
july_29 = july_29.groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
july_29['checkin_date'] = 'July 29'
july_29['checkout_date'] = 'August 5'
july_29['checkin_mon'] = 7
july_29['checkin_day'] = 29

C:\Users\bswil\AppData\Local\Temp\ipykernel_25068\1812611800.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  may_27 = may_27.groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
C:\Users\bswil\AppData\Local\Temp\ipykernel_25068\1812611800.py:17: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  june_24 = june_24.groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()
C:\Users\bswil\AppData\Local\Temp\ipykernel_25068\1812611800.py:28: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  july_29 = july_29.groupby(['location'])['temp', 'tempmax', 'tempmin', 'days_with_rain_10_years'].mean().reset_index()


In [30]:
july_22

,location,temp,tempmax,tempmin,days_with_rain_10_years,checkin_date,checkout_date,checkin_mon,checkin_day
0,Dauphin Island,80.82500,88.09500,74.55625,6.875,July 22,July 29,7,22
1,Gulfport Beach,82.52500,90.15000,75.98375,6.125,July 22,July 29,7,22
2,Isle of Palms,81.58750,89.08375,75.08125,5.250,July 22,July 29,7,22
3,Miramar Beach,83.59125,89.62500,77.91875,6.125,July 22,July 29,7,22
4,Orange Beach,81.36625,88.28750,75.03375,6.750,July 22,July 29,7,22
5,Panama City Beach,81.71375,87.06750,76.34500,6.750,July 22,July 29,7,22
6,Tybee Island,81.89000,90.49250,75.02000,5.750,July 22,July 29,7,22


In [18]:
check_in_weather = pd.merge(august_12, august_19,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, august_26,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, august_5,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, july_1,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, july_15,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, july_22,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, july_29,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, july_8,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, june_10,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, june_17,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, june_24,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, june_3,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, march_11,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, may_13,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, may_20,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, may_27,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, may_6,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')
check_in_weather = pd.merge(check_in_weather, october_7,
                            on = ('location', 'temp', 'tempmax', 'tempmin', 'days_with_rain_10_years',
                                   'checkin_date', 'checkout_date', 'checkin_mon', 'checkin_day'), how='outer')

In [19]:
check_in_weather

,location,temp,tempmax,tempmin,days_with_rain_10_years,checkin_date,checkout_date,checkin_mon,checkin_day
0,Dauphin Island,80.18500,87.92500,74.03375,7.375,August 12,August 19,8,12
1,Gulfport Beach,81.94125,89.74250,75.61500,6.875,August 12,August 19,8,12
2,Isle of Palms,80.92250,88.37250,74.69000,5.375,August 12,August 19,8,12
3,Miramar Beach,83.12000,88.81750,77.62125,7.000,August 12,August 19,8,12
4,Orange Beach,80.55750,87.68500,74.38000,7.125,August 12,August 19,8,12
...,...,...,...,...,...,...,...,...,...
128,Isle of Palms,72.26625,80.47000,65.61000,35.000,October 7,October 14,10,7
129,Miramar Beach,76.84000,84.49375,69.63750,23.750,October 7,October 14,10,7
130,Orange Beach,73.99750,83.12125,65.68750,23.750,October 7,October 14,10,7
131,Panama City Beach,75.59250,83.50375,68.15500,32.500,October 7,October 14,10,7


# Merging Checkin Weather with Listings

In [20]:
master_list = pd.merge(all_locations, check_in_weather,
        on = 'location', how = 'outer')
master_list

,bedrooms,type,sleeps,pets_allowed,rating,review_count,cost_night,latitude,longitude,bathrooms,location,temp,tempmax,tempmin,days_with_rain_10_years,checkin_date,checkout_date,checkin_mon,checkin_day
0,3,House,9,True,4.888889,36,282,30.360629,-89.114292,2.0,Gulfport Beach,81.941250,89.742500,75.615000,6.875,August 12,August 19,8,12
1,3,House,9,True,4.888889,36,282,30.360629,-89.114292,2.0,Gulfport Beach,81.965000,89.602500,75.587500,6.000,August 19,August 26,8,19
2,3,House,9,True,4.888889,36,282,30.360629,-89.114292,2.0,Gulfport Beach,81.118333,88.498333,74.568333,5.500,August 26,September 1,8,26
3,3,House,9,True,4.888889,36,282,30.360629,-89.114292,2.0,Gulfport Beach,82.757500,90.486250,76.093750,6.000,August 5,August 12,8,5
4,3,House,9,True,4.888889,36,282,30.360629,-89.114292,2.0,Gulfport Beach,82.161250,89.570000,75.711250,5.250,July 1,July 8,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42194,2,Villa,8,False,4.800000,20,275,32.115299,-80.846573,2.0,Tybee Island,73.491250,82.493750,64.901250,3.625,May 13,May 13,5,13
42195,2,Villa,8,False,4.800000,20,275,32.115299,-80.846573,2.0,Tybee Island,76.607500,85.347500,68.745000,3.875,May 20,May 20,5,20
42196,2,Villa,8,False,4.800000,20,275,32.115299,-80.846573,2.0,Tybee Island,77.911250,86.603750,70.240000,5.500,May 27,June 3,5,27
42197,2,Villa,8,False,4.800000,20,275,32.115299,-80.846573,2.0,Tybee Island,71.987500,82.403750,62.112500,2.375,May 6,May 13,5,6


In [21]:
for index, row in master_list.iterrows():
    if row.bedrooms < 3:
        master_list.loc[index, 'beds'] = '2 or Less'
    elif row.bedrooms > 5:
        master_list.loc[index, 'beds'] = '6 or more'
    else:
        master_list.loc[index, 'beds'] = '3-5'

In [22]:
for index, row in master_list.iterrows():
    if row.type == 'Apartment':
        master_list.loc[index, 'listing_type'] = 'Apartment'
    elif row.type == 'Bed and Breakfast':
        master_list.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Boat':
        master_list.loc[index, 'listing_type'] = 'Other'
    elif row.type == 'Bungalow':
        master_list.loc[index, 'listing_type'] = 'Apartment'
    elif row.type == 'Cabin':
        master_list.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Condo':
        master_list.loc[index, 'listing_type'] = 'Condo'
    elif row.type == 'Cottage':
        master_list.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Country house/chateau':
        master_list.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Estate':
        master_list.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Hotel':
        master_list.loc[index, 'listing_type'] = 'Hotel'
    elif row.type == 'Hotel suite':
        master_list.loc[index, 'listing_type'] = 'Hotel'
    elif row.type == 'House':
        master_list.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Lodge':
        master_list.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Mobile home':
        master_list.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Recreational vehicle':
        master_list.loc[index, 'listing_type'] = 'Other'
    elif row.type == 'Resort':
        master_list.loc[index, 'listing_type'] = 'Hotel'
    elif row.type == 'Studio':
        master_list.loc[index, 'listing_type'] = 'Apartment'
    elif row.type == 'Townhome':
        master_list.loc[index, 'listing_type'] = 'House'
    else:
        master_list.loc[index, 'listing_type'] = 'Condo'

In [23]:
for index, row in master_list.iterrows():
    if row.pets_allowed == True:
        master_list.loc[index, 'pets_allowed'] = 'Yes'
    else:
        master_list.loc[index, 'pets_allowed'] = 'No'

In [26]:
for index, row in master_list.iterrows():
    if row.location == 'Tybee Island':
        master_list.loc[index, 'travel_time'] = 'Over 7.5 Hours'
    elif row.location == 'Isle of Palms':
        master_list.loc[index, 'travel_time'] = 'Over 7.5 Hours'
    else:
        master_list.loc[index, 'travel_time'] = 'Under 7.5 Hours'

In [27]:
master_list

,bedrooms,type,sleeps,pets_allowed,rating,review_count,cost_night,latitude,longitude,bathrooms,...,tempmax,tempmin,days_with_rain_10_years,checkin_date,checkout_date,checkin_mon,checkin_day,beds,listing_type,travel_time
0,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,89.742500,75.615000,6.875,August 12,August 19,8,12,3-5,House,Under 7.5 Hours
1,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,89.602500,75.587500,6.000,August 19,August 26,8,19,3-5,House,Under 7.5 Hours
2,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,88.498333,74.568333,5.500,August 26,September 1,8,26,3-5,House,Under 7.5 Hours
3,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,90.486250,76.093750,6.000,August 5,August 12,8,5,3-5,House,Under 7.5 Hours
4,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,89.570000,75.711250,5.250,July 1,July 8,7,1,3-5,House,Under 7.5 Hours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42194,2,Villa,8,No,4.800000,20,275,32.115299,-80.846573,2.0,...,82.493750,64.901250,3.625,May 13,May 13,5,13,2 or Less,Condo,Over 7.5 Hours
42195,2,Villa,8,No,4.800000,20,275,32.115299,-80.846573,2.0,...,85.347500,68.745000,3.875,May 20,May 20,5,20,2 or Less,Condo,Over 7.5 Hours
42196,2,Villa,8,No,4.800000,20,275,32.115299,-80.846573,2.0,...,86.603750,70.240000,5.500,May 27,June 3,5,27,2 or Less,Condo,Over 7.5 Hours
42197,2,Villa,8,No,4.800000,20,275,32.115299,-80.846573,2.0,...,82.403750,62.112500,2.375,May 6,May 13,5,6,2 or Less,Condo,Over 7.5 Hours


In [35]:
master_list.loc[master_list['checkin_date'] != 'October 7']

,bedrooms,type,sleeps,pets_allowed,rating,review_count,cost_night,latitude,longitude,bathrooms,...,tempmax,tempmin,days_with_rain_10_years,checkin_date,checkout_date,checkin_mon,checkin_day,beds,listing_type,travel_time
0,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,89.742500,75.615000,6.875,August 12,August 19,8,12,3-5,House,Under 7.5 Hours
1,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,89.602500,75.587500,6.000,August 19,August 26,8,19,3-5,House,Under 7.5 Hours
2,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,88.498333,74.568333,5.500,August 26,September 1,8,26,3-5,House,Under 7.5 Hours
3,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,90.486250,76.093750,6.000,August 5,August 12,8,5,3-5,House,Under 7.5 Hours
4,3,House,9,Yes,4.888889,36,282,30.360629,-89.114292,2.0,...,89.570000,75.711250,5.250,July 1,July 8,7,1,3-5,House,Under 7.5 Hours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42193,2,Villa,8,No,4.800000,20,275,32.115299,-80.846573,2.0,...,71.848750,51.532500,35.000,March 11,March 18,3,11,2 or Less,Condo,Over 7.5 Hours
42194,2,Villa,8,No,4.800000,20,275,32.115299,-80.846573,2.0,...,82.493750,64.901250,3.625,May 13,May 13,5,13,2 or Less,Condo,Over 7.5 Hours
42195,2,Villa,8,No,4.800000,20,275,32.115299,-80.846573,2.0,...,85.347500,68.745000,3.875,May 20,May 20,5,20,2 or Less,Condo,Over 7.5 Hours
42196,2,Villa,8,No,4.800000,20,275,32.115299,-80.846573,2.0,...,86.603750,70.240000,5.500,May 27,June 3,5,27,2 or Less,Condo,Over 7.5 Hours


In [36]:
updated_listings = pd.read_csv('../data/updated_listings.csv')

In [38]:
updated_listings = updated_listings.drop(columns = 'Unnamed: 0')

In [41]:
updated_listings

,bedrooms,type,sleeps,pets_allowed,rating,review_count,cost_night,latitude,longitude,image_url,bathrooms,checkin_month,checkin_day,location
0,3,House,8,False,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,3.0,10,7,Dauphin Island
1,4,House,8,False,5.000000,3,316,30.250673,-88.145703,https://media.vrbo.com/lodging/79000000/785900...,4.0,10,7,Dauphin Island
2,3,House,8,False,4.777778,18,200,30.252405,-88.145048,https://media.vrbo.com/lodging/64000000/635000...,2.0,10,7,Dauphin Island
3,2,Condo,6,False,4.875000,8,145,30.260590,-88.112778,https://media.vrbo.com/lodging/47000000/469900...,1.0,10,7,Dauphin Island
4,2,Condo,6,False,5.000000,1,136,30.244760,-88.098330,https://media.vrbo.com/lodging/84000000/837300...,2.5,10,7,Dauphin Island
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,5,House,12,False,0.000000,0,878,32.004870,-80.847920,https://media.vrbo.com/lodging/93000000/923600...,3.0,10,7,Tybee Island
2780,4,House,15,False,0.000000,0,366,31.998015,-80.847574,https://media.vrbo.com/lodging/92000000/917100...,3.0,10,7,Tybee Island
2781,5,Condo,10,False,0.000000,0,1218,31.993702,-80.849484,https://media.vrbo.com/lodging/93000000/925900...,4.0,10,7,Tybee Island
2782,6,Condo,12,False,0.000000,0,1337,31.993887,-80.849234,https://media.vrbo.com/lodging/93000000/925900...,5.0,10,7,Tybee Island


In [42]:
for index, row in updated_listings.iterrows():
    if row.bedrooms < 3:
        updated_listings.loc[index, 'beds'] = '2 or Less'
    elif row.bedrooms > 5:
        updated_listings.loc[index, 'beds'] = '6 or more'
    else:
        updated_listings.loc[index, 'beds'] = '3-5'

In [46]:
for index, row in updated_listings.iterrows():
    if row.type == 'Apartment':
        updated_listings.loc[index, 'listing_type'] = 'Apartment'
    elif row.type == 'Bed and Breakfast':
        updated_listings.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Boat':
        updated_listings.loc[index, 'listing_type'] = 'Other'
    elif row.type == 'Bungalow':
        updated_listings.loc[index, 'listing_type'] = 'Apartment'
    elif row.type == 'Cabin':
        updated_listings.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Condo':
        updated_listings.loc[index, 'listing_type'] = 'Condo'
    elif row.type == 'Cottage':
        updated_listings.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Country house/chateau':
        updated_listings.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Estate':
        updated_listings.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Hotel':
        updated_listings.loc[index, 'listing_type'] = 'Hotel'
    elif row.type == 'Hotel suite':
        updated_listings.loc[index, 'listing_type'] = 'Hotel'
    elif row.type == 'House':
        updated_listings.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Lodge':
        updated_listings.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Mobile home':
        updated_listings.loc[index, 'listing_type'] = 'House'
    elif row.type == 'Recreational vehicle':
        updated_listings.loc[index, 'listing_type'] = 'Other'
    elif row.type == 'Resort':
        updated_listings.loc[index, 'listing_type'] = 'Hotel'
    elif row.type == 'Studio':
        updated_listings.loc[index, 'listing_type'] = 'Apartment'
    elif row.type == 'Townhome':
        updated_listings.loc[index, 'listing_type'] = 'House'
    else:
        updated_listings.loc[index, 'listing_type'] = 'Condo'

In [48]:
for index, row in updated_listings.iterrows():
    if row.pets_allowed == True:
        updated_listings.loc[index, 'pets_allowed'] = 'Yes'
    else:
        updated_listings.loc[index, 'pets_allowed'] = 'No'

In [49]:
for index, row in updated_listings.iterrows():
    if row.location == 'Tybee Island':
        updated_listings.loc[index, 'travel_time'] = 'Over 7.5 Hours'
    elif row.location == 'Isle of Palms':
        updated_listings.loc[index, 'travel_time'] = 'Over 7.5 Hours'
    else:
        updated_listings.loc[index, 'travel_time'] = 'Under 7.5 Hours'

In [50]:
updated_listings

,bedrooms,type,sleeps,pets_allowed,rating,review_count,cost_night,latitude,longitude,image_url,bathrooms,checkin_month,checkin_day,location,beds,listing_type,travel_time
0,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,3.0,10,7,Dauphin Island,3-5,House,Under 7.5 Hours
1,4,House,8,No,5.000000,3,316,30.250673,-88.145703,https://media.vrbo.com/lodging/79000000/785900...,4.0,10,7,Dauphin Island,3-5,House,Under 7.5 Hours
2,3,House,8,No,4.777778,18,200,30.252405,-88.145048,https://media.vrbo.com/lodging/64000000/635000...,2.0,10,7,Dauphin Island,3-5,House,Under 7.5 Hours
3,2,Condo,6,No,4.875000,8,145,30.260590,-88.112778,https://media.vrbo.com/lodging/47000000/469900...,1.0,10,7,Dauphin Island,2 or Less,Condo,Under 7.5 Hours
4,2,Condo,6,No,5.000000,1,136,30.244760,-88.098330,https://media.vrbo.com/lodging/84000000/837300...,2.5,10,7,Dauphin Island,2 or Less,Condo,Under 7.5 Hours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,5,House,12,No,0.000000,0,878,32.004870,-80.847920,https://media.vrbo.com/lodging/93000000/923600...,3.0,10,7,Tybee Island,3-5,House,Over 7.5 Hours
2780,4,House,15,No,0.000000,0,366,31.998015,-80.847574,https://media.vrbo.com/lodging/92000000/917100...,3.0,10,7,Tybee Island,3-5,House,Over 7.5 Hours
2781,5,Condo,10,No,0.000000,0,1218,31.993702,-80.849484,https://media.vrbo.com/lodging/93000000/925900...,4.0,10,7,Tybee Island,3-5,Condo,Over 7.5 Hours
2782,6,Condo,12,No,0.000000,0,1337,31.993887,-80.849234,https://media.vrbo.com/lodging/93000000/925900...,5.0,10,7,Tybee Island,6 or more,Condo,Over 7.5 Hours


In [51]:
updated_master_list = pd.merge(updated_listings, check_in_weather,
        on = 'location', how = 'outer')
updated_master_list

,bedrooms,type,sleeps,pets_allowed,rating,review_count,cost_night,latitude,longitude,image_url,...,listing_type,travel_time,temp,tempmax,tempmin,days_with_rain_10_years,checkin_date,checkout_date,checkin_mon,checkin_day_y
0,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,...,House,Under 7.5 Hours,80.18500,87.92500,74.033750,7.375000,August 12,August 19,8,12
1,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,...,House,Under 7.5 Hours,80.12500,87.70000,74.161250,6.750000,August 19,August 26,8,19
2,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,...,House,Under 7.5 Hours,79.66000,87.65000,73.223333,5.833333,August 26,September 1,8,26
3,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,...,House,Under 7.5 Hours,80.89500,88.36375,74.635000,7.375000,August 5,August 12,8,5
4,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,...,House,Under 7.5 Hours,80.13125,87.26250,73.870000,6.375000,July 1,July 8,7,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52891,6,Condo,18,Yes,0.000000,0,2126,31.998451,-80.848333,https://media.vrbo.com/lodging/93000000/929600...,...,Condo,Over 7.5 Hours,73.49125,82.49375,64.901250,3.625000,May 13,May 13,5,13
52892,6,Condo,18,Yes,0.000000,0,2126,31.998451,-80.848333,https://media.vrbo.com/lodging/93000000/929600...,...,Condo,Over 7.5 Hours,76.60750,85.34750,68.745000,3.875000,May 20,May 20,5,20
52893,6,Condo,18,Yes,0.000000,0,2126,31.998451,-80.848333,https://media.vrbo.com/lodging/93000000/929600...,...,Condo,Over 7.5 Hours,77.91125,86.60375,70.240000,5.500000,May 27,June 3,5,27
52894,6,Condo,18,Yes,0.000000,0,2126,31.998451,-80.848333,https://media.vrbo.com/lodging/93000000/929600...,...,Condo,Over 7.5 Hours,71.98750,82.40375,62.112500,2.375000,May 6,May 13,5,6


In [54]:
updated_master_list = updated_master_list[['bedrooms', 'type', 'sleeps', 'pets_allowed', 'rating', 'review_count',
       'cost_night', 'latitude', 'longitude', 'image_url', 'bathrooms', 'location', 'beds', 'listing_type',
       'travel_time', 'temp', 'tempmax', 'tempmin', 'checkin_date', 'checkout_date']]

In [56]:
updated_master_list

,bedrooms,type,sleeps,pets_allowed,rating,review_count,cost_night,latitude,longitude,image_url,bathrooms,location,beds,listing_type,travel_time,temp,tempmax,tempmin,checkin_date,checkout_date
0,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,3.0,Dauphin Island,3-5,House,Under 7.5 Hours,80.18500,87.92500,74.033750,August 12,August 19
1,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,3.0,Dauphin Island,3-5,House,Under 7.5 Hours,80.12500,87.70000,74.161250,August 19,August 26
2,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,3.0,Dauphin Island,3-5,House,Under 7.5 Hours,79.66000,87.65000,73.223333,August 26,September 1
3,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,3.0,Dauphin Island,3-5,House,Under 7.5 Hours,80.89500,88.36375,74.635000,August 5,August 12
4,3,House,8,No,4.795454,44,242,30.255512,-88.137262,https://media.vrbo.com/lodging/49000000/488500...,3.0,Dauphin Island,3-5,House,Under 7.5 Hours,80.13125,87.26250,73.870000,July 1,July 8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52891,6,Condo,18,Yes,0.000000,0,2126,31.998451,-80.848333,https://media.vrbo.com/lodging/93000000/929600...,4.0,Tybee Island,6 or more,Condo,Over 7.5 Hours,73.49125,82.49375,64.901250,May 13,May 13
52892,6,Condo,18,Yes,0.000000,0,2126,31.998451,-80.848333,https://media.vrbo.com/lodging/93000000/929600...,4.0,Tybee Island,6 or more,Condo,Over 7.5 Hours,76.60750,85.34750,68.745000,May 20,May 20
52893,6,Condo,18,Yes,0.000000,0,2126,31.998451,-80.848333,https://media.vrbo.com/lodging/93000000/929600...,4.0,Tybee Island,6 or more,Condo,Over 7.5 Hours,77.91125,86.60375,70.240000,May 27,June 3
52894,6,Condo,18,Yes,0.000000,0,2126,31.998451,-80.848333,https://media.vrbo.com/lodging/93000000/929600...,4.0,Tybee Island,6 or more,Condo,Over 7.5 Hours,71.98750,82.40375,62.112500,May 6,May 13
